## Procesado de Ficheros II
---


### 2. Tablas
---
Las formatos más simples de datos que nos encontraremos consistirán en hojas de cálculo, archivos CSV o bases de datos. 

Los datos estarán dispuestos en tablas donde cada columna contiene una pieza de información y cada registro o fila se corresponde con una muestra (_sample_) del _dataset_.

Las columnas pueden contener tanto datos numéricos como texto. Sin embargo, los tensores de PyTorch sólo pueden contener valores numéricos, por lo que será necesaria cierta conversión.

#### 2.1 Usando datos reales
---
Para nuestro ejemplo, vamos a usar un _dataset_ con información acerca de la composición química de una serie de muestras de _vinho verde_ del norte de Portugal.

El fichero contiene una colección de muestras, organizadas en 12 columnas. Las 11 primeras columnas contienen medidas relacionadas con la composición química de la muestra, y la última columna una calificación del vino desde 0 (muy malo) a 10 (excelente).

Las 12 columnas son las siguientes:

- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulphates
- alcohol
- quality

Una posible aplicación de ML sería la de predecir la calidad del vino a partir de su caracterización química.

<br>

![](data/image-lect/wine_predict.png)

<br>


#### 2.2 Cargando los datos
---
Antes de que podamos procesar los datos con PyTorch es necesario que los carguemos de alguna manera. En Python tenemos las siguientes opciones:

- El módulo _csv_ de Python
- Pandas
- NumPy

Habitualmente emplearemos Pandas para realizar este trabajo. Sin embargo, debido a la magnífica interoperabilidad de PyTorch y NumPy, vamos a emplear éste último (en este caso).


In [39]:
import csv
import numpy as np

wine_path = "data/tabular/winequality-white.csv"

# Leemos el archivo usando ; como delimitador y omitiendo la fila de encabezados (skiprows)
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

# column names
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

Ahora podemos convertir el array de NumPy en un tensor de PyTorch

In [5]:
import torch

wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

### 2.3 Tipos de valores
---
A la hora de procesar los datos, debemos tener en cuenta cuál es su naturaleza y escala de medida, es decir, si son valores **continuos**, **ordinales** o **nominales** (categóricos).

Los valores **continuos** están ordenados y la diferencia (separación) entre ello es significativa. Por ejemplo, si decimos que el paquete A es 2kg más pesado que el paquete B, la diferencia tiene un significado fijo, con independencia de que A pese 5 ó 500 kg. En general, los valores que tengan asociado una unidad, son continuos.

Los valores **ordinales** están ordenados pero no existen relaciones fijas entre ellos. Por ejemplo, si hacemos una clasificación de tamaños XS, S, M, L, XL, a la que le asignamos los valores de 1 a 5, sabemos que algo de tamaño 3 (M) es más grande que algo de tamaño 1 (XS), pero no "cuánto". Es importante recordar que no podemos **"operar matemáticamente"** con estos valores. Son simples clasificaciones de orden. En nuestro ejemplo, tratar de calcular la media de los tamaños no tendría ningún sentido.

Por último, los valores **categóricos** (discretos o nominales) ni presentan ordenación ni tienen significado numérico. Por ejemplo, la asignación de los valores 1 al _agua_, 2 al _café_, 3 a la _leche_ y 4 a un _refresco_ no aporta ninguna información de tipo numérico (orden o valor)

#### 2.3.1 Representando la puntuación
---
En nuestro caso, podemos tratar la puntuación (_quality_) del vino como un valor continuo y realizar la predicción mediante un proceso de **regresión lineal**, o tratarla como un valor nominal (_label_) y realizar la predicción mediante una **regresión logística o clasificación**.

Tanto en un caso como en otro, eliminaremos la columna de la puntuación del _dataset_ de entrada y la mantendremos en un tensor aparte.

In [7]:
data = wineq[:, :-1]
print("input data:", data.shape)

target = wineq[:, -1]
print("target:", target.shape)

input data: torch.Size([4898, 11])
target: torch.Size([4898])


Si quisiéramos transformar nuestro tensor _target_ en un tensor de _etiquetas_ (para hacer una regresión logística) tenemos dos opciones:

La más simple es convertir sus valores a un valor entero (podríamos hacer lo mismo si fueran eqtiquetas de texto):


In [8]:
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

#### 2.3.2 One-hot encoding
---
La otra opción es contruir una _codificación one-hot_ de las puntuaciones. Consiste en construir, por cada puntuación, un vector binario con tantos elementos como puntuaciones posibles (en nuestro caso 11, de 0 a 10) incializados a 0 salvo para la columna de la puntuación correspondiente, que estará a 1. 

Por ejemplo, una puntuación de **5** se correspondería con el vector: \[0,0,0,0,0,**1**,0,0,0,0,0\]

Fíjate que el hecho de que la puntuación corresponda con el índice del vector es puramente circunstancial y no implica nada desde la perspectiva de la clasificación. Podríamos reordenar las columnas de dicho vector sin ninguna repercusión en la clasificación. Lo único importante es saber con qué columna se corresponde cada puntuación (_label_).

El que nos decantemos por una opción u otra puede depender de ciertas cuestiones relacionadas con la naturaleza de la puntuación. El emplear un valor numérico discreto (opción 1) induce una ordenación en las puntuaciones y cierta noción de "distancia" entre ellas, que pueden ser representativo para nosotros. Por otro lado, si la puntuación es puramente discreta o nominal, la codificación _one-hot_ es más apropiada al no inducir orden o distancia.

En PyTorch, podemos crear nuestro tensor de codificación _one-hot_ usando el método **scatter_**, que rellena el tensor con los valores de un tensor origen a lo largo de los índices indicados como argumentos:



In [14]:
target_onehot = torch.zeros(target.shape[0], 10)
print(target_onehot.shape)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)
target_onehot

torch.Size([4898, 10])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

El método **scatter_**, por cada fila del nuevo tensor (*target_onehot*), lee la puntuación correspondiente del tensor _target_ (convertido a formato columna y con las mismas dimensiones que *target_onehot* al hacer _unsqueeze(1)_) y usa su valor cómo índice para establecer el valor indicado (1.0 en este caso) en la columna correspondiente de *target_onehot*.

PyTorch nos permite usar índices de clases directamente como _targets_ cuando entrenamos redes neuronales. Sin embargo, si quisiéramos utilizar la puntuación como entrada nominal de la red, tendremos que convertirla en un tensor _one-hot_.

#### 2.3.3 Valores ordinales
---
Ya hemos visto como tratar datos continuos y nominales. En el caso de datos ordinales no existe una receta simple para tratar con ellos. En general, podremos tratarlos como valores categóricos (perdiendo la información de ordenación) o como continuos (introduciendo cierta noción de distancia).

<br>

![](data/image-lect/ordinal_values.png)

<br>

Volvamos a nuestro tensor con los datos conteniendo las 11 variables asociadas a los análisis químicos de los vinos y obtengamos la media y desviación estándar de cada columna:

In [26]:
data_mean = torch.mean(data, dim=0)
print("mean = ", data_mean, "\nshape = ", data_mean.shape)

data_var = torch.std(data, dim=0)
print("\nvar = ", data_var, "\nshape = ", data_var.shape)

mean =  tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01]) 
shape =  torch.Size([11])

var =  tensor([8.4387e-01, 1.0079e-01, 1.2102e-01, 5.0721e+00, 2.1848e-02, 1.7007e+01,
        4.2498e+01, 2.9909e-03, 1.5100e-01, 1.1413e-01, 1.2306e+00]) 
shape =  torch.Size([11])


En este caso, _dim=0_ indica que la reducción se realizará a lo largo de la dimensión 0. Ahora, podemos normalizar los datos restando la media y dividiendo por la desviación estándar

In [30]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.5808e-01, -2.5958e-02,  7.4188e-02,  ..., -4.8449e-01,
         -1.1795e-01, -1.5453e+00],
        [-6.0393e-01,  6.8536e-02,  1.6697e-02,  ...,  2.8754e-01,
          4.5344e-04, -9.1430e-01],
        [ 1.3555e+00,  5.5402e-03,  1.8917e-01,  ...,  1.8460e-01,
         -1.4755e-01, -3.7344e-01],
        ...,
        [-3.8622e-01, -1.2045e-01, -4.1449e-01,  ..., -5.1022e-01,
         -8.8350e-02, -1.0044e+00],
        [-1.4748e+00,  3.7038e-02, -9.8286e-02,  ...,  3.9047e-01,
         -3.2516e-01,  2.0605e+00],
        [-9.3051e-01, -2.1495e-01,  1.3168e-01,  ...,  1.8460e-01,
         -5.0277e-01,  1.1590e+00]])

#### 2.3.4 Buscando umbrales
---
Finalmente, podemos hacer algunos análisis directos sobre los datos con objeto de hacer alguna asunción o predicción simple sobre los mismos.

Por ejemplo, vamos a buscar aquellas muestras cuya puntuación sea igual o inferior a 3:

In [32]:
bad_indexes = target <= 3
print(bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum())
print(bad_indexes)

torch.Size([4898]) torch.bool tensor(20)
tensor([False, False, False,  ..., False, False, False])


Obtendremos un tensor de _bool_, de la misma dimension que _target_, con un valor _True_ para aquellas filas (_samples_) que cumplan la condición. En nuestro caso, 20.

Usando este nuevo tensor, podremos indexar _data_ para filtrar las muestras (_samples_) correspondientes

In [35]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

Vamos ahora a "dividir" nuestras muestras en tres categorías diferentes (bad, mid, good) y obtener la media de cada una de las columnas

In [53]:
bad_data = data[target<=3]
mid_data = data[(target>3) & (target<7)]
good_data = data[target>=7]

print(f"bad: {bad_data.shape[0]}; mid: {mid_data.shape[0]}; good: {good_data.shape[0]}")

bad_mean = bad_data.mean(dim=0)
mid_mean = mid_data.mean(dim=0)
good_mean = good_data.mean(dim=0)

print(f"\n{'PROPERTY':^25s}  {'BAD':^6s} {'MID':^6s} {'GOOD':^6s}")
for i, name in enumerate(col_list[:-1]):
    print(f"{i:2d} {col_list[i]:<22s} {bad_mean[i]:>6.2f} {mid_mean[i]:>6.2f} {good_mean[i]:>6.2f}")


bad: 20; mid: 3818; good: 1060

        PROPERTY            BAD    MID    GOOD 
 0 fixed acidity            7.60   6.89   6.73
 1 volatile acidity         0.33   0.28   0.27
 2 citric acid              0.34   0.34   0.33
 3 residual sugar           6.39   6.71   5.26
 4 chlorides                0.05   0.05   0.04
 5 free sulfur dioxide     53.33  35.42  34.55
 6 total sulfur dioxide   170.60 141.83 125.25
 7 density                  0.99   0.99   0.99
 8 pH                       3.19   3.18   3.22
 9 sulphates                0.47   0.49   0.50
10 alcohol                 10.34  10.26  11.42


De este análisis simplista podríamos deducir que hay una correspondencia entre valores elevados de dióxido de sulfuro y una baja calidad del vino.

Utilizando este criterio, vamos a tratar de encontrar vinos de alta calidad. Para ello, vamos a utilizar como umbral el valor medio de los vinos de calidad media.

In [55]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6] # extraemos la columna correspondiente
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

Según esto, entorno a la mitad de los vinos serían de alta calidad. 

Vamos a buscar los índices de los vinos realmente buenos:

In [56]:
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

Nos salen unos 500 más que los predichos.

Finalmente, vamos a comparar nuestra predicción con el ranking real, a ver qué tal...

In [64]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item() # cuántas filas coinciden?
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)

Tenemos unos 2000 vinos bien! 

Sobre un 74% de nuestras predicciones son correctas (vinos que decimos que son buenos y lo son realmente). Sin embargo, sólo identificamos el 62% de los buenos vinos reales.

Como se puede apreciar, una buena predicción está influenciada por muchas más variables que no hemos tenido en cuenta. 